## Setup

In [2]:
!pip install nibabel
!pip install pandas
!pip install nilearn
!pip install sklearn
!pip install matplotlib
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your depende

In [3]:
## Load packages

import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.utils import Bunch
from nilearn import image
import os

In [4]:
## do you want the func 

def get_subject_id(sub_dir_name):
    return sub_dir_name.split('-')[1]

def get_subject_dirs(data_dir, exclude_subjects_prefix, n_subjects=None):
    # Ensure only directories are considered as subject directories
    subject_dirs = [
        os.path.join(data_dir, sub_dir) for sub_dir in os.listdir(data_dir)
        if sub_dir.startswith('sub-') and os.path.isdir(os.path.join(data_dir, sub_dir))
        and not any(get_subject_id(sub_dir).startswith(exclude) for exclude in exclude_subjects_prefix)
    ]

    if n_subjects:
        subject_dirs = subject_dirs[:n_subjects]
    
    return subject_dirs

def get_func_files(subject_dirs):
    func_files = []
    for sub_dir in subject_dirs:
        func_dir = os.path.join(sub_dir, 'func')
        if os.path.isdir(func_dir):
            func_files.extend([
                os.path.join(func_dir, f) for f in os.listdir(func_dir) 
                if f.endswith('.nii.gz') and 'echo' in f
            ])
    return func_files





In [5]:
#data_dir_2021 = '/work/FaceWord_fMRI/BIDS/derivatives' 
data_dir_2022 = '/work/FaceWord_fMRI/BIDS_2022/derivatives'  
data_dir_2023 = '/work/FaceWord_fMRI/BIDS_2023/derivatives'
data_dir_2024 = '/work/FaceWord_fMRI/BIDS_2024/derivatives'


#exclude_subjects_prefix_2021 = ['0054', '0077', '0078', '0079', '0080', '0081', '0083', '0084', '0090','0091', '0092', '0093', '0094', '0096', '0097', '0098', '0099', '0100', '0101', '0103', '0104', '0105', '0106', '0107']
exclude_subjects_prefix_2022 = ['0054', '0057', '0059', '0060', '0062', '0064', '0065', '0066', '0090', '0091', '0079', '0058']
exclude_subjects_prefix_2023 = ['0057', '0058', '0059','0060', '0062', '0064', '0065', '0066', '0077', '0097']
exclude_subjects_prefix_2024 = ['0127', '0130']

n_subjects = None  # Or specify an integer, e.g., 10

# Get subject directories
#subject_dirs_21 = get_subject_dirs(data_dir_2021, exclude_subjects_prefix_2021, n_subjects)
subject_dirs_22 = get_subject_dirs(data_dir_2022, exclude_subjects_prefix_2022, n_subjects)
subject_dirs_23 = get_subject_dirs(data_dir_2023, exclude_subjects_prefix_2023, n_subjects)
subject_dirs_24 = get_subject_dirs(data_dir_2024, exclude_subjects_prefix_2024, n_subjects)

# Get functional files
#func_21 = get_func_files(subject_dirs_21)
func_22 = get_func_files(subject_dirs_22)
func_23 = get_func_files(subject_dirs_23)
func_24 = get_func_files(subject_dirs_24)

In [6]:
func =  func_22 + func_23 + func_24

In [7]:
import os
import glob

def get_subject_id(file_path):
    return file_path.split('/')[-3].split('-')[1]

def get_filtered_files(pattern, exclude_subjects):
    files = glob.glob(pattern)
    filtered_files = [
        file for file in files 
        if get_subject_id(file) not in exclude_subjects
    ]
    return filtered_files

# Define the pattern for the files you want to fetch
#pattern_21 = '/work/FaceWord_fMRI/BIDS/sub-*/func/*events.tsv'
pattern_22 = '/work/FaceWord_fMRI/BIDS_2022/sub-*/func/*events.tsv'
pattern_23 = '/work/FaceWord_fMRI/BIDS_2023/sub-*/func/*events.tsv'
pattern_24 = '/work/FaceWord_fMRI/BIDS_2024/sub-*/func/*events.tsv'



# Get the filtered list of event file paths
#event_paths_21 = get_filtered_files(pattern_21, exclude_subjects_prefix_2021)
event_paths_22 = get_filtered_files(pattern_22, exclude_subjects_prefix_2022)
event_paths_23 = get_filtered_files(pattern_23, exclude_subjects_prefix_2023)
event_paths_24 = get_filtered_files(pattern_24, exclude_subjects_prefix_2024)

In [8]:
event_paths =  event_paths_22 + event_paths_23 + event_paths_24

## Loading in files

In [9]:
# Initialize an empty list to store the DataFrames
dataframes = []

# Iterate over the list of files
for file in event_paths:
    # Read the TSV file into a DataFrame
    df = pd.read_csv(file, sep='\t')
    # Append the DataFrame to the list
    dataframes.append(df)

# Now you have a list of DataFrames, one for each TSV file
for i, df in enumerate(dataframes):
    print(f"DataFrame {i}:\n", df.head(), "\n")


DataFrame 0:
        onset  duration trial_type  response_time       word  gender  age  sub  \
0   7.313593  0.689985  image_neg       0.773779  hierarchy  female    1   81   
1  14.726210  0.677254  image_pos       0.561213       corn  female    1   81   
2  27.327418  0.676393  image_neg       0.443441     banker  female    1   81   
3  37.326404  0.677234  image_pos       0.611027    stapler  female    1   81   
4  44.718290  0.685535  image_pos       0.519143      water  female    1   81   

               task  run  year  month  day  hour  minute  
0  EPIsequencewords    5  2022      4   20    10      59  
1  EPIsequencewords    5  2022      4   20    10      59  
2  EPIsequencewords    5  2022      4   20    10      59  
3  EPIsequencewords    5  2022      4   20    10      59  
4  EPIsequencewords    5  2022      4   20    10      59   

DataFrame 1:
        onset  duration trial_type  response_time      word  gender  age  sub  \
0   7.305710  0.689794  image_pos       1.423578 

In [10]:
from nilearn import plotting
from nilearn.image import mean_img

#plotting.view_img(mean_img(fmri_filename), threshold=None)

In [11]:
# Faces

for df in dataframes:
    # Creating the 'valence' column based on 'trial_type' values
    df['faces'] = df['trial_type'].apply(lambda x: 1 if x in ['image_pos', 'image_neg'] else 0)


In [12]:

for i in range(len(dataframes)):
    # Keeping only the 'valence' and 'onset' columns
    dataframes[i] = dataframes[i][['faces', 'onset']]

In [13]:
#CONCAT

from nilearn import image
import numpy as np
import pandas as pd

# Assuming 'func' is a list of file paths to your nii.gz files
# Assuming 'dataframes' is a list of dataframes corresponding to each subject

processed_data = []

# Load the first image to use its affine as the reference
reference_img = image.load_img(func[0])

for i, f in enumerate(func):
    # Load the functional MRI data for the current subject
    fmri_img = image.load_img(f)
    
    # Resample the image to the reference affine
    resampled_img = image.resample_img(fmri_img, target_affine=reference_img.affine, target_shape=reference_img.shape[:3])

    # Get the events (onsets of stimuli) for the current subject
    events = dataframes[i]["onset"].values  # Ensure onsets are in numpy array format
    
    # Define the number of time points in your data
    n_volumes = resampled_img.shape[-1]
    
    # Get the repetition time (TR) from the image header
    TR = fmri_img.header.get_zooms()[-1]
    
    # Create a boolean mask based on event timing
    mask = np.zeros(n_volumes, dtype=bool)
    for event in events:
        time_point = int(event / TR)  # Assuming TR is the repetition time
        if time_point < n_volumes:
            mask[time_point] = True
    
    # Apply the mask to your data
    masked_data = image.index_img(resampled_img, mask)
    
    # Convert masked_data to numpy array and append to the list
    processed_data.append(masked_data.get_fdata())

# Concatenate all numpy arrays along the time axis
all_data = np.concatenate(processed_data, axis=-1)

# Optionally, convert the concatenated data back to a NIfTI image
# Use the affine from the reference image
concatenated_img = nib.Nifti1Image(all_data, reference_img.affine)

nib.save(concatenated_img, '/work/exam/concatenated_img_yes.nii')


KeyboardInterrupt: 

In [ ]:
concatenated_img = nib.load("/work/exam/concatenated_img_yes.nii")

In [ ]:
# Convert for faces

cond = pd.concat(dataframes)
cond_array = cond['faces'].values

In [ ]:
from nilearn import datasets, image
from nilearn.input_data import MultiNiftiMasker
from nilearn.image import math_img

# Fetch the atlas
#multiscale = datasets.fetch_atlas_basc_multiscale_2015(data_dir='/work/Home/work')
#atlas_filename = multiscale.scale064

# Apply the threshold to the atlas image
#thresholded_atlas_img = image.threshold_img(atlas_filename, threshold="30%")

# Binarize the thresholded image to create a binary mask
#binary_mask_img = math_img('img > 0', img=thresholded_atlas_img)

# Create the MultiNiftiMasker with the binary mask
#maskermasker = MultiNiftiMasker(mask_img=binary_mask_img, standardize=True,
                                #memory='nilearn_cache', resampling_target="data",
                                #detrend=True, verbose=1)

In [ ]:
##BINARIZE

from nilearn import datasets, image
from nilearn.input_data import MultiNiftiMasker
from nilearn.image import math_img

# Fetch the atlas
#multiscale = datasets.fetch_atlas_basc_multiscale_2015(data_dir='/work/Home/work')
#atlas_filename = multiscale.scale064

#BINAR
#binary_atlas_img = image.binarize_img(atlas_filename, threshold="30%")

#RESAMPLING
#resampled_binary_atlas_img = image.resample_img(binary_atlas_img, target_affine=reference_img.affine, target_shape=reference_img.shape[:3])

# Create the MultiNiftiMasker with the binary mask
#maskermasker = MultiNiftiMasker(mask_img=binary_atlas_img, standardize=True,
                                #memory='nilearn_cache', resampling_target="data",
                                #detrend=True, verbose=1)

In [ ]:
## This is 80/20 split training
from nilearn.image import index_img
from nilearn.decoding import Decoder

split_index = int(0.8 * len(cond_array))

# Check the shape of the concatenated_img
data_shape = concatenated_img.shape
total_length = data_shape[-1]

# Calculate the 80% split index
split_index_img = int(0.8 * total_length)

masked_data_train = index_img(concatenated_img, slice(0, split_index_img))
masked_data_test = index_img(concatenated_img, slice(split_index_img, total_length))
conditions_train = cond_array[:split_index]
conditions_test = cond_array[split_index:]

batch_size = 500
decoder = Decoder(
    estimator="svc", standardize="zscore_sample", n_jobs = -1)
decoder.fit(masked_data_train, conditions_train)

prediction = decoder.predict(masked_data_test)

# The prediction accuracy is calculated on the test data: this is the accuracy
# of our model on examples it hasn't seen to examine how well the model perform
# in general.

predicton_accuracy = (prediction == conditions_test).sum() / float(
    len(conditions_test))
print(f"Prediction Accuracy: {predicton_accuracy:.3f}")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


# Generate confusion matrix
conf_matrix = confusion_matrix(conditions_test, prediction)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)


plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Oranges")
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix of SVC model for faces and words - 80/20 training-test split')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score

if hasattr(decoder, "decision_function"):
    decision_scores = decoder.decision_function(masked_data_test)
    roc_auc = roc_auc_score(conditions_test, decision_scores)
else:
    # If using a classifier that supports predict_proba
    probabilities = decoder.predict_proba(masked_data_test)[:, 1]
    roc_auc = roc_auc_score(conditions_test, probabilities)

# Calculate F1 score
f1 = f1_score(conditions_test, prediction)

# Print scores
print(f"ROC AUC: {roc_auc:.3f}")
print(f"F1 Score: {f1:.3f}")